In [1]:
from plotly.subplots import make_subplots
import numpy as np
import plotly.graph_objects as go
import plotly
import networkx as nx
import scipy.sparse as sp
import scipy as sc
import scipy as sc
import time
import pandas as pd
from genpr.tensorflow.io_planetoid_type import load_data
from genpr.data.io import load_dataset
import tqdm
from sklearn.preprocessing import normalize
from scipy.spatial import distance
from sklearn.metrics import average_precision_score, accuracy_score

In [84]:
adj, attr_matrix, y_train, y_val, y_test, train_mask, val_mask, test_mask, ally, allx = load_data('citeseer')

In [424]:
adj

<3327x3327 sparse matrix of type '<class 'numpy.longlong'>'
	with 9228 stored elements in Compressed Sparse Row format>

In [85]:
X = attr_matrix.toarray()

In [327]:
AHAT = calc_A_hat(Af, 1)
#rex = np.identity(nnodes) - alpha * AHAT 
max(abs(np.linalg.eig(     AHAT /9   )[0]))

0.8084638094052876

In [350]:
AHAT = calc_A_hat(Af, 1)
alpha = 0.9

In [357]:
zp = np.ones(AHAT.shape[0])#/AHAT.shape[0]

In [360]:
for i in range(1000):
    zp = zp@ (alpha*AHAT) 
    zp = zp/np.linalg.norm(zp) 
    print(max(abs(np.array(zp*np.linalg.norm(zp) )[0])))

0.16520809207746165
0.16520809207746145
0.16520809207746162
0.1652080920774617
0.16520809207746173
0.16520809207746173
0.1652080920774618
0.16520809207746176
0.1652080920774618
0.1652080920774618
0.1652080920774618
0.1652080920774618
0.1652080920774618
0.1652080920774618
0.1652080920774618
0.1652080920774618
0.1652080920774618
0.1652080920774618
0.1652080920774618
0.1652080920774618
0.1652080920774618
0.1652080920774618
0.1652080920774618
0.1652080920774618
0.1652080920774618
0.1652080920774618
0.1652080920774618
0.1652080920774618
0.1652080920774618
0.1652080920774618
0.1652080920774618
0.1652080920774618
0.1652080920774618
0.1652080920774618
0.1652080920774618
0.1652080920774618
0.1652080920774618
0.1652080920774618
0.1652080920774618
0.1652080920774618
0.1652080920774618
0.1652080920774618
0.1652080920774618
0.1652080920774618
0.1652080920774618
0.1652080920774618


KeyboardInterrupt: 

In [342]:
np.array(zp)

array([[0.0149257 , 0.01720664, 0.01494842, ..., 0.0129724 , 0.01491637,
        0.00803567]])

In [331]:
np.linalg.norm(AHAT /9)

3.84707184705551

In [245]:
def exact_pr(A,b):
    return  np.linalg.inv(A )@ b

In [476]:
max(abs(np.linalg.eig(   0.9* MMd  )[0]))

0.3541972845475299

In [479]:
max(abs(np.linalg.eig(     0.9* AP + 0.9* MMd  )[0]))

1.2329673580766596

In [478]:
0.9 + 0.3541972845475299

1.25419728454753

In [483]:
max(np.diag(MMd))*0.9

0.005954631379962193

In [472]:
alpha = 0.9
AHAT,MMd, AP = calc_A_hat(Af, 1)


rex = (np.identity(nnodes)  - alpha * AHAT )#@D_m  
start = timeit.default_timer()
Z = exact_pr(rex, (1-alpha) * y_train )
stop = timeit.default_timer()

print('Time: ', stop - start) 
real_acc = accuracy_score( np.argmax(y_test[test_mask], axis=-1), 
                          np.argmax(np.array(Z )[test_mask], axis=-1))
 
reval_al_acc = accuracy_score( np.argmax(y_val[val_mask], axis=-1), 
                              np.argmax(np.array( Z )[val_mask], axis=-1))
 
print(reval_al_acc, real_acc)

Time:  1.3334762279991992
0.708 0.736


In [ ]:
MMd

In [272]:
Af = adj.toarray()

In [88]:
nnodes = Af.shape[0]

In [459]:
MMz = np.zeros(MMx.shape)
count_v = 0
q7 = np.percentile(abs(MMx), 75)
for i in tqdm.tqdm(range(nnodes)):
    
    for j in range(nnodes):
        if abs(MMx[i,j]) >= q7:
            count_v += 2
            MMz[i,j] = abs(MMx[i,j]) 
            MMz[j,i] = abs(MMx[i,j])
    


100%|██████████| 3327/3327 [00:10<00:00, 311.44it/s]


In [458]:
q7

0.0005401026194977045

In [461]:
len(MMz[MMz !=0])

4339858

In [ ]:
np.percentile()

In [457]:
#q9 = np.percentile(abs(MMx), 0.9)
q7 = np.percentile(abs(MMx), 75)

In [429]:
nnodes **2

11068929

In [433]:
count_v

4034192

In [413]:
len(np.where(MMz > 0.00027005130974885227)[0])

4339858

In [387]:
np.median(abs(MMx))

0.00027005130974885227

In [373]:
3327*3327

11068929

In [465]:
def calc_A_hat(adj_matrix: sp.spmatrix, delta ) -> sp.spmatrix:
    nnodes = adj_matrix.shape[0]
    A = adj_matrix  + sp.eye(nnodes)#Ω#@ D_invsqrt_corr
    D_vec = np.sum(A, axis=1).A1 
    D_vec_invsqrt_corr = 1 /  D_vec  
    D_invsqrt_corr = sp.diags(D_vec_invsqrt_corr) 
    
     
    return      D_invsqrt_corr@A + delta*(  MMz )@D_invsqrt_corr, (  MMz )@D_invsqrt_corr   , D_invsqrt_corr@A

In [329]:
Xn = X.T
Xn = Xn  
MMx =  np.dot(Xn.T, Xn)/(X.shape[1]  )

In [471]:
Z =  y_train# np.ones(y_train.shape) / y_train.shape[1]
alpha = 0.9
AHAT, mmp, tty =  calc_A_hat(Af,  1) 
#rex =  
for i in range(20):
    #nr = np.linalg.norm(rex@Z)
    Z =   alpha * AHAT  @Z   + (1-alpha) *y_train 
    
     
    print(accuracy_score( np.argmax(y_test[test_mask], axis=-1), 
                              np.argmax(np.array( Z)[test_mask], axis=-1))) 

0.504
0.479
0.449
0.422
0.394
0.362
0.338
0.322
0.301
0.274
0.263
0.254
0.247
0.243
0.24
0.237
0.237
0.236
0.232
0.231


In [28]:
import timeit

start = timeit.default_timer()
Z= np.linalg.solve(rex, (1-alpha)*y_train)
stop = timeit.default_timer()

print('Time: ', stop - start) 

Time:  0.25812195299999985


In [11]:
alpha = 0.9
AHAT = calc_A_hat(Af,1)
rex = np.identity(nnodes) - alpha * AHAT 

In [ ]:
start = timeit.default_timer()
Z = np.linalg.lstsq(rex, (1-alpha)*y_train)
stop = timeit.default_timer()

print('Time: ', stop - start) 

/Users/mikhail/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning:

`rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.



In [14]:
import timeit



#Your statements here

 

In [402]:
 accuracy_score( np.argmax(y_test[test_mask], axis=-1), 
                          np.argmax(np.array(Z)[test_mask], axis=-1))

0.699

In [36]:
alpha = 0.9
AHAT = calc_A_hat(Af, 0.1)
rex = np.identity(nnodes) - alpha * AHAT 
start = timeit.default_timer()
Z = exact_pr(rex, (1-alpha)*y_train )
stop = timeit.default_timer()

print('Time: ', stop - start) 
real_acc = accuracy_score( np.argmax(y_test[test_mask], axis=-1), 
                          np.argmax(np.array(Z)[test_mask], axis=-1))
 
reval_al_acc = accuracy_score( np.argmax(y_val[val_mask], axis=-1), 
                              np.argmax(np.array(Z)[val_mask], axis=-1))
 
print(reval_al_acc, real_acc)

Time:  0.6975666369999658
0.67 0.701


In [10]:
Dd = np.sum(MMx, axis=1)
Dd = sp.diags(Dd)

In [26]:
M = X.T@Z@np.linalg.inv((Z.T@Z))

In [30]:
X_ = M@Z.T

In [32]:
X_.shape

(1433, 2708)

In [42]:
X_n = X - np.mean(X, axis=-1)

ValueError: operands could not be broadcast together with shapes (2708,1433) (2708,) 

In [29]:
real_acc = accuracy_score( np.argmax(y_test[test_mask], axis=-1), 
                          np.argmax(np.array(M)[test_mask], axis=-1))
 
reval_al_acc = accuracy_score( np.argmax(y_val[val_mask], axis=-1), 
                              np.argmax(np.array(M)[val_mask], axis=-1))
 
print(reval_al_acc, real_acc)

IndexError: boolean index did not match indexed array along dimension 0; dimension is 1433 but corresponding boolean dimension is 2708

In [95]:
(Z@M.T@M).shape

(2708, 7)

In [35]:
X.shape

(2708, 1433)

In [36]:
M 

matrix([[ 0.04545767,  0.10053375,  0.05775564, ...,  0.03425495,
          0.05433316,  0.00372982],
        [-0.00743137,  0.41774077,  0.04615568, ...,  0.00976749,
          0.17398157,  0.05486394],
        [ 0.28400247,  0.39402449,  0.36422142, ...,  0.099485  ,
          0.11930487,  0.01533129],
        ...,
        [ 0.08695902,  0.02168185, -0.0009923 , ...,  0.00325138,
          0.0062296 ,  0.00106313],
        [ 0.25932973,  0.31556864,  0.08413436, ...,  0.17474019,
          0.38986164,  0.04554033],
        [ 0.00112873,  0.08634125, -0.01619744, ...,  0.08309962,
         -0.00646274, -0.00196771]])

In [427]:
from sklearn.decomposition import PCA
 


In [429]:
XX.shape[1]

3703

In [430]:
MMx.shape

(3327, 3327)

In [431]:
Af.shape

(2708, 2708)

In [1109]:
XX = attr_matrix.toarray()
Xm = XX - np.mean(XX) 
pca = PCA(n_components=10)
X_p = pca.fit_transform(Xm)
MMx =  np.dot(XX,XX.T )#/

In [534]:
MMx =  np.dot(XX,XX.T )/(XX.shape[1] -1)

In [492]:
In = np.diag(np.ones(Af.shape[0] ))

In [494]:
Inl = np.zeros(In.shape)#[train_mask] * 0

In [495]:
Inl[train_mask,train_mask] =1

In [496]:
sum(np.diag(Inl))

140.0

In [497]:
train_mask

array([ True,  True,  True, ..., False, False, False])

In [498]:
alphas = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

In [1118]:
test_acc = [] 
val_acc = [] 

nnodes = y_train.shape[0]
for alpha in [0.5]:  
    for delta in [11]:
        AHAT = calc_A_hat(Af, delta)
        print(alpha, delta)
        rex = np.identity(nnodes) - alpha * AHAT  #+  (1-alpha)*MMx
        exact_solution_dwd = exact_pr(rex, (1-alpha)*y_train )
        real_acc = accuracy_score( np.argmax(y_test[test_mask], axis=-1), 
                                  np.argmax(np.array(exact_solution_dwd)[test_mask], axis=-1))
        test_acc.append(real_acc)
        reval_al_acc = accuracy_score( np.argmax(y_val[val_mask], axis=-1), 
                                      np.argmax(np.array(exact_solution_dwd)[val_mask], axis=-1))
        val_acc.append(reval_al_acc)
        print(reval_al_acc, real_acc)

0.5 11
0.118 0.14


In [387]:
d = 1000
n = 200
c1 = 0.5
c2 = 0.4

In [388]:
c1*n/d

0.1

In [417]:
mean1 =  np.ones(d)
cov1 =  np.diag(np.ones(d))*c1*n/d
x1 = np.random.multivariate_normal(mean1, cov1, int(n/2)) 

In [418]:
mean2 =  np.ones(d) * 0.5
cov2 =  np.diag(np.ones(d) ) * 0.1/d
x2 = np.random.multivariate_normal(mean2, cov2, int(n/2)) 

In [419]:
np.linalg.norm(cov1)

3.1622776601683804

In [420]:
print(np.linalg.norm(np.array(cov1) - np.array(cov2))) 
print(np.linalg.norm(np.array(mean1) - np.array(mean2))) 
XT = np.concatenate([x1, x2 ])

0.6324555320336762
15.811388300841896


In [421]:
from sklearn.decomposition import PCA
 
pca = PCA(n_components=np.min(XT.shape),svd_solver='full' )
pca.fit(XT.T)

PCA(copy=True, iterated_power='auto', n_components=200, random_state=None,
    svd_solver='full', tol=0.0, whiten=False)

In [422]:
len(pca.components_[1])

200

In [424]:
cols = plotly.colors.DEFAULT_PLOTLY_COLORS  
 
x = np.arange(len(pca.components_[1]))
fig = make_subplots(rows=1, cols=1, subplot_titles=( r'$ \alpha  = 0.1$'))
error_type = 0  

fig.add_trace(go.Scatter(x=x, y= pca.components_[0], mode='lines', 
                             line=dict(width=2, color=cols[1]),
                             name=r'$exact $', showlegend=True,), 
             row=1, col=1, )
    
fig.update_xaxes(title_text=r'$ time $' )  
 
  
fig.update_layout(height=800, width=1050, title_text=r'$Political \ blogs \ 2 \ classes \ \alpha = 0.1 $')
fig.show()

In [195]:
np.mean(pca.components_[-1])

0.016708041116080617

In [144]:
pca.explained_variance_

array([1.73623195e+01, 1.18775158e+01, 8.81733403e-31])

In [146]:
pca.mean_.shape

(40,)

In [1040]:
pca.components_[-1]

array([ 0.13394398,  0.42647498, -0.55545226, -0.08083382,  0.03073945,
       -0.01211318, -0.00798079, -0.13734148,  0.30257857, -0.08859381,
        0.07903899,  0.17619363, -0.03432436,  0.35347553, -0.06398778,
        0.01077527,  0.06941114,  0.19152251,  0.10709628,  0.00407688,
       -0.02961454, -0.03527319,  0.02917416, -0.04941497,  0.00549625,
        0.04361507,  0.06377062,  0.03593976, -0.0727716 , -0.01275028,
       -0.07498477, -0.218548  ,  0.09980962,  0.10978365, -0.18504611,
        0.01654128,  0.03534736, -0.03839176, -0.07973518,  0.08902629])

In [1057]:
np.mean(vals)

16.470722256669177

In [555]:
from sklearn.metrics import plot_confusion_matrix

In [ ]:
plot_confusion_matrix()

In [554]:
pca.components_

array([[ 0.01507055, -0.13720814,  0.03172919, -0.33265766,  0.08495205,
        -0.1077813 ,  0.15523234,  0.11860562,  0.01323912, -0.11501172,
         0.005042  , -0.30884361,  0.08937252, -0.07024408, -0.29077692,
         0.00469065,  0.04860928,  0.2129703 ,  0.17716969, -0.07157724,
         0.04567975, -0.20412903, -0.37063638,  0.05488781, -0.15481161,
         0.18888029, -0.35151688,  0.04066287,  0.0457275 , -0.17180744,
        -0.11102647, -0.01213078, -0.13957573, -0.19822759,  0.10601204,
         0.12626104, -0.00155573, -0.11489867, -0.08218627, -0.05239125],
       [ 0.03617036, -0.3261071 ,  0.07944817, -0.05896337,  0.0770415 ,
         0.18834431,  0.18880365,  0.04397219, -0.10114689, -0.16635927,
        -0.08862768,  0.49678918, -0.10393224, -0.10556133, -0.14936803,
         0.08669769, -0.17570949, -0.27350254,  0.19169457, -0.02223326,
         0.00875745, -0.10158794, -0.09136646, -0.04348482,  0.16877123,
        -0.237834  , -0.15948427, -0.16524712, -0.

In [556]:
cols = plotly.colors.DEFAULT_PLOTLY_COLORS  
exact_nomalized = np.ones(300000)  * real_acc
x = np.arange(30000)
fig = make_subplots(rows=1, cols=1, subplot_titles=( r'$ \alpha  = 0.1$'))
error_type = 0
error_name =['accuracy', '||AX-b||']
 
fig.add_trace(go.Scatter(x=alphas, y=val_acc, mode='lines', 
                             line=dict(width=2, color=cols[0]),
                             name=r'$exact $', showlegend=True,), 
             row=1, col=1, )  
   
   

    
fig.update_xaxes(title_text=r'$ time $' )
fig.update_xaxes(type="log", row=1, col=1)
fig.update_yaxes(  type="log", row=1, col=1) 
 
  
fig.update_layout(height=800, width=1050, title_text=r'$Political \ blogs \ 2 \ classes \ \alpha = 0.1 $')
fig.show()

In [65]:
val_acc

[0.73, 0.73, 0.73, 0.73, 0.73, 0.73, 0.73, 0.73, 0.73, 0.73]